<a href="https://colab.research.google.com/github/NINAD-ML/PRODIGY_ML_04/blob/main/gesture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Download the dataset from Kaggle
import kagglehub

# Download latest version of the dataset
path = kagglehub.dataset_download("gti-upm/leapgestrecog")
print("Path to dataset files:", path)

# Import necessary libraries
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Set dataset directory.
# Adjust this based on how the dataset is extracted.
# For example, if the dataset structure is:
#   path/
#       leapGestRecog/
#           00/
#           01_palm/
#           02_l/
data_dir = os.path.join(path, "leapGestRecog")

# Define parameters for image loading and augmentation
img_height, img_width = 64, 64
batch_size = 32

# Use ImageDataGenerator to perform data augmentation and split the dataset
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,         # Reserve 20% for validation
    rotation_range=20,
    zoom_range=0.15,
    horizontal_flip=True
)

# Create training generator
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

# Create validation generator
validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Build a simple CNN model for gesture recognition
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model with appropriate loss and optimizer
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Print model summary to verify architecture
model.summary()

# Setup callbacks for early stopping and model checkpointing
early_stop = EarlyStopping(monitor='val_loss', patience=5)
checkpoint = ModelCheckpoint('gesture_model.h5', monitor='val_accuracy', save_best_only=True)

# Train the model
epochs = 20
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Save the final model after training
model.save('gesture_model_final.h5')
print("Model training complete and saved!")


100%|██████████| 2.13G/2.13G [00:22<00:00, 103MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/gti-upm/leapgestrecog/versions/1
Found 16000 images belonging to 10 classes.
Found 4000 images belonging to 10 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

...

500/500 ━━━━━━━━━━━━━━━━━━━━ 178s 356ms/step - accuracy: 0.9398 - loss: 0.1451 - val_accuracy: 0.7465 - val_loss: 2.9235
Epoch 6/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 328ms/step - accuracy: 0.9513 - loss: 0.1270

500/500 ━━━━━━━━━━━━━━━━━━━━ 186s 372ms/step - accuracy: 0.9513 - loss: 0.1270 - val_accuracy: 0.7875 - val_loss: 2.1256
Epoch 7/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 315ms/step - accuracy: 0.9535 - loss: 0.1138

500/500 ━━━━━━━━━━━━━━━━━━━━ 179s 358ms/step - accuracy: 0.9535 - loss: 0.1138 - val_accuracy: 0.7912 - val_loss: 2.2846
Epoch 8/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 180s 359ms/step - accuracy: 0.9637 - loss: 0.0932 - val_accuracy: 0.7340 - val_loss: 2.4546
Epoch 9/20
500/500 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.9661 - loss: 0.0894

500/500 ━━━━━━━━━━━━━━━━━━━━ 184s 367ms/step - accuracy: 0.9661 - loss: 0.0894 - val_accuracy: 0.7997 - val_loss: 1.9951


Model training complete and saved!


In [4]:
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("/content/gesture_model_final.h5")

# Compile the model again (use the same optimizer and loss function as before)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Model recompiled successfully!")



Model recompiled successfully!


In [5]:
loss, accuracy = model.evaluate(validation_generator)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")
print(f"Validation Loss: {loss:.4f}")


125/125 ━━━━━━━━━━━━━━━━━━━━ 22s 176ms/step - accuracy: 0.7972 - loss: 1.9647
Validation Accuracy: 80.85%
Validation Loss: 1.9787


In [1]:
from tensorflow.keras.preprocessing import image
import numpy as np
import tensorflow as tf

# Load the trained model
model = tf.keras.models.load_model("/content/gesture_model_final.h5")
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Path to the test image
img_path = "/content/download.jpg"  # Change this to your actual image file

# Load and preprocess the image
img = image.load_img(img_path, target_size=(64, 64))  # Resize to the same size used during training
img_array = image.img_to_array(img) / 255.0  # Normalize pixel values
img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

# Make a prediction
predictions = model.predict(img_array)
predicted_class = np.argmax(predictions)

# Get class labels (from your training generator, ensure this matches your dataset)
# If you have a 'train_generator' used during model training, get the class labels as follows:
# class_labels = list(train_generator.class_indices.keys())

# If you don't have 'train_generator', define your class labels manually like this:
class_labels = ['Palm', 'L', 'Fist', 'Peace', 'Thumb Up', 'Heart', 'OK', 'V', 'Flat', 'One Finger']

# Output the predicted gesture name
print(f"Predicted Gesture: {class_labels[predicted_class]}")

# Display class labels and their corresponding indices
for index, label in enumerate(class_labels):
    print(f"Index {index}: Gesture '{label}'")


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = '/content/gesture_model_final.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Adjust the directory path to the one where your dataset is located
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/sample_data/dataset',  # Update with the path where your dataset is located
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/sample_data/dataset',  # Update with the path where your dataset is located
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 20 images belonging to 3 classes.
Found 20 images belonging to 3 classes.


In [18]:
train_generator = train_datagen.flow_from_directory(
    '/content/sample_data/dataset/train',  # Correct path to your training images
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/sample_data/dataset/val',  # Correct path to your validation images
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 10 images belonging to 1 classes.
Found 10 images belonging to 1 classes.


In [19]:
import os

# Check the content of your directories
train_dir = '/content/sample_data/dataset/train'
val_dir = '/content/sample_data/dataset/val'

# List the contents of train and validation directories
print("Train Directory:", os.listdir(train_dir))
print("Validation Directory:", os.listdir(val_dir))


Train Directory: ['heart']
Validation Directory: ['heart']


In [20]:
train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/sample_data/dataset/train',  # Path to training dataset
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = val_datagen.flow_from_directory(
    '/content/sample_data/dataset/val',  # Path to validation dataset
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)


Found 10 images belonging to 1 classes.
Found 10 images belonging to 1 classes.
